In [17]:
import numpy as np
import pandas as pd
import csv
import json
import time

from datetime import date, datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

In [26]:
base_url = 'https://www.booking.com/'

mac_path = '/Users/admin/Downloads/chromedriver'
comcenter_path = r'C:/Users/6142923026/Downloads/chromedriver.exe'

In [8]:
class chk:
    def __init__(self, cin, cout):
        self.cin = cin
        self.cout = cout

In [9]:
def chk_inout():
    
    chk_list = []
    N_DAYS = 3
    STR_LEN = len('2019-02-09')
    
    t_date = datetime.now()
    
    for i in range(4,4+N_DAYS):
        
        chkin = (t_date + timedelta(days=i))
        chkout = (t_date + timedelta(days=i+1))
        
        chkin = chkin.isoformat()[:10]
        chkout = chkout.isoformat()[:10]
        
        chk_list.append(chk(chkin, chkout))

    return chk_list

In [10]:
chk_list = chk_inout()

for c in chk_list:
    print(c.cin, c.cout)

2019-03-01 2019-03-02
2019-03-02 2019-03-03
2019-03-03 2019-03-04


In [20]:
def get_search_url(chk_list):
    
    s_results = []
    
    for chk in chk_list:

        driver = webdriver.Chrome(comcenter_path)
        driver.get(base_url)

        assert "Booking.com" in driver.title
        
        chkin = chk.cin
        chkout = chk.cout

        time.sleep(3)

        elem = driver.find_element_by_name("ss")

        elem.clear()
        elem.send_keys("Bangkok")

        calendar_xpath = "//div[@class='xp__dates-inner xp__dates__checkin']"
        date_1_xpath = "//td[@class='bui-calendar__date' and @data-date='" + chkin + "']"
        date_2_xpath = "//td[@class='bui-calendar__date' and @data-date='" + chkout + "']"

        chkin_elem = driver.find_element_by_xpath(calendar_xpath).click()
        time.sleep(2)

        chkin_date_elem = driver.find_element_by_xpath(date_1_xpath).click()
        time.sleep(1)

        chkout_date_elem = driver.find_element_by_xpath(date_2_xpath).click()
        elem.send_keys(Keys.RETURN)

        time.sleep(3)
        driver.implicitly_wait(2)

        all_pages_xpath = "//li[@class='bui-pagination__item sr_pagination_item']"
        all_pages = driver.find_elements_by_xpath(all_pages_xpath)
#         print(all_pages)
        last_page = int(all_pages[-1].text)
        print(last_page)
        
        hname_path = "//a[@class='hotel_name_link url']"
        hname_list = driver.find_elements_by_xpath(hname_path)
        hlink = [ele.get_attribute('href') for ele in hname_list]
        
        if(len(s_results)==0):
            s_results = hlink
        else:
            s_results = np.hstack((s_results, hlink))
        
        for i in range(2,last_page):
            print(i)
            next_page_btn_xpath = "//li[@class='bui-pagination__item bui-pagination__next-arrow']"
            next_page_btn = driver.find_element_by_xpath(next_page_btn_xpath).click()
            
            time.sleep(5)
            driver.implicitly_wait(2)

            hname_list = driver.find_elements_by_xpath(hname_path)
            hlink = [ele.get_attribute('href') for ele in hname_list]

            if(len(s_results)==0):
                s_results = hlink
            else:
                s_results = np.hstack((s_results, hlink))
            
    return s_results

In [22]:
res_url = get_search_url(chk_list)

50
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [24]:
with open('booking-urls.txt','w') as f:
    for url in res_url:
        f.write(url + '\n')
    f.close()

In [31]:
def get_hotel_table(soup, i):

    h_dict = dict(hid = [], 
                  hname = [], 
                  hlocation = [], 
                  hstars = [], 
                  hratings = [],
                  hnearby = [],
                  hfacilities = [],
                  hcheckin = [],
                  hcheckout = []
                 )

    hotel_name = soup.find('h2', class_='hp__hotel-name').getText().strip('\n').split('\n')[1]
    hotel_stars = soup.find(class_='bk-icon-wrapper bk-icon-stars star_track')
    if(hotel_stars):
        hstars = hotel_stars.getText().strip('\n')
    else:
        hstars = None

    hotel_loc = soup.find(class_='hp_address_subtitle')
    if(hotel_loc):
        hlocation = hotel_loc.getText().strip('\n')
    else:
        hlocation = None

    review_score = soup.find(class_='bui-review-score')
    score_string = review_score.getText().split(' ')
    ratings = [s for s in score_string if(s!='')][0]

    nearby_str = ''
    
    nearby_blk = soup.find(class_='property_page_surroundings_block')
    if(nearby_blk):
        nearby_pl = nearby_blk.find_all(class_='bui-list__item')

        for p in nearby_pl:
            pl_ = p.getText().replace('\n',';')
            near_pl_list = pl_.split(';')
            near_pl_list = [pp for pp in near_pl_list if(pp!='')]
            places = near_pl_list[0]
            distance = near_pl_list[-1]
            p_d = ' '.join([places, distance]) + '; '

            nearby_str = ''.join([nearby_str, p_d])

    facil_blk = soup.find(class_='hp_desc_important_facilities').getText().replace('\n',';')
    facility_list = facil_blk.split(';')
    facility_list = [f for f in facility_list if(f!='')][1:]

    facil = '; '.join(facility_list)

    chkindate = soup.find('div', {'data-placeholder':'Check-in Date'}).getText().replace('\n','')
    chkoutdate = soup.find('div', {'data-placeholder':'Check-out Date'}).getText().replace('\n','')

    h_dict['hid'] = i
    h_dict['hname'].append(hotel_name)
    h_dict['hstars'].append(hstars)
    h_dict['hlocation'].append(hlocation)
    h_dict['hratings'].append(ratings)
    h_dict['hnearby'].append(nearby_str)
    h_dict['hfacilities'].append(facil)

    h_dict['hcheckin'].append(chkindate)
    h_dict['hcheckout'].append(chkoutdate)
    
    df_hotel = pd.DataFrame.from_dict(h_dict)

    return df_hotel

In [29]:
def get_room_table(soup, i):
    r_dict = dict(hid = [],
                  roomName = [],
                  bedType = [],
                  capacity = [],
                  rackPrice = [],
                  price = [],
                  breakfast = [],
                  features = [])

    r_dict['hid'] = i
    roomtypes = soup.find_all(class_='hprt-roomtype-icon-link')
    
    noneText = None

    if(roomtypes):
        for r in roomtypes:
            if(r):
                room_name = r.getText().strip('\n')
                r_dict['roomName'].append(room_name)

                r_parent = r.parent.parent.parent.parent.parent

                r_capacity = r_parent.find(class_='hprt-table-cell-occupancy')
                if(r_capacity):
                    r_cap = r_capacity.getText().strip('\n')
                    r_cap = r_cap[-1]
                else:
                    r_cap = noneText
                r_dict['capacity'].append(r_cap)
                
                r_bedtype = r_parent.find(class_='bed-types-wrapper')
                if(r_bedtype):
                    r_btype = r_bedtype.getText().replace('\n',';').split(';')
                    btype_list = [bt for bt in r_btype if(bt!='')]
                    btype = '; '.join(btype_list)
                else:
                    btype = noneText
                r_dict['bedType'].append(btype)

                r_features = r_parent.find(class_='hprt-facilities-block')
                if(r_features):
                    feature_list = r_features.getText().replace('\n',';').split(';')
                    f_list = [f.strip(' •') for f in feature_list if(f!='')]
                    features = '; '.join(f_list)
                else:
                    features = noneText
                r_dict['features'].append(features)

                rack_prices = []
                prices = []

                price_block = r_parent.findNext('td', class_='hprt-table-cell-price')
                if(price_block):

                    price = price_block.find(class_='hprt-price-price')
                    pr = price.getText().replace('\n','')
                    prices.append(pr)

                    rack_rate_price = price_block.find(class_='hp-blue-sans-rack-rate')
                    if(rack_rate_price):
                        rack_pr = rack_rate_price.getText().replace('\n','')
                    else:
                        rack_pr = pr
                    rack_prices.append(rack_pr)

                    bf_icon = r_parent.find(class_='hprt-conditions')
                    if(bf_icon):
                        bf_txt_list = bf_icon.getText().replace('\n',';').split(';')
                        bf_txt = '; '.join([b for b in bf_txt_list if(b!='')])
                    else:
                        bf_txt = noneText

                    r_dict['rackPrice'].append(rack_pr)
                    r_dict['price'].append(pr)
                    r_dict['breakfast'].append(bf_txt)

                    
                    price_blk_2 = price_block.findNext('td', class_='hprt-table-cell-price')

                    if(price_blk_2):
                        price_2 = price_blk_2.find(class_='hprt-price-price')
                        pr_2 = price_2.getText().replace('\n','')

                        rack_rate_price_2 = price_blk_2.find(class_='hp-blue-sans-rack-rate')
                        if(rack_rate_price_2):
                            rack_pr_2 = rack_rate_price_2.getText().replace('\n','')
                        else:
                            rack_pr_2 = pr_2

                        if(bf_icon):
                            bf_icon_2 = bf_icon.findNext(class_='hprt-conditions')
                            if(bf_icon_2):
                                bf2_list = bf_icon_2.getText().replace('\n',';').split(';')
                                bf2_txt = '; '.join([b for b in bf2_list if(b!='')])

                            r_dict['roomName'].append(room_name)
                            r_dict['capacity'].append(r_cap)
                            r_dict['bedType'].append(btype)
                            r_dict['features'].append(features)
                            r_dict['rackPrice'].append(rack_pr_2)
                            r_dict['price'].append(pr_2)
                            r_dict['breakfast'].append(bf2_txt)
                    
                else:
                    
                    r_dict['rackPrice'].append(noneText)
                    r_dict['price'].append(noneText)
                    r_dict['breakfast'].append(noneText)

    df_room = pd.DataFrame.from_dict(r_dict)

    return df_room

In [32]:
df_all_hotel = pd.DataFrame()
df_all_room = pd.DataFrame()

with open('booking-urls.txt','r') as f:
    lines = [line.rstrip('\n') for line in f]
    for i in range(len(lines)):
        print(i)
        driver = webdriver.Chrome(comcenter_path)
        driver.get(lines[i])
        raw_html = driver.execute_script("return document.documentElement.outerHTML")

        soup = BeautifulSoup(raw_html,'html.parser')

        df_hotel = get_hotel_table(soup, i)
        df_room = get_room_table(soup, i)

        df_all_hotel = df_all_hotel.append(df_hotel)
        df_all_room = df_all_room.append(df_room)

0
1
2
3


AttributeError: 'NoneType' object has no attribute 'getText'

In [310]:
df_all_hotel.to_csv('all_hotel_booking.csv')
df_all_hotel

,hid,hname,hlocation,hstars,hratings,hnearby,hfacilities,hcheckin,hcheckout
0,0,Issara by D Hostel,"125 Fuang Nakhon Road, Phranakhon, 10200 Bangk...",None,9.2,Thai Chamber of commerce 0.2 km; Wat Ratchapra...,Non-smoking rooms; WiFi in all areas; Free WiF...,"\nWednesday, February 20, 2019\n","\nThursday, February 21, 2019\n"
0,1,Solo Express Sukhumvit 81,"120 Soi Sukhumvit 81, Phra Khanong , 10260 Ban...",3-star hotel,7.9,BTS-On Nut 0.5 km; BTS-Bang Chak 1.1 km; W Dis...,Non-smoking rooms; Family rooms; Free parking;...,"Wednesday, February 20, 2019","Thursday, February 21, 2019"
0,2,Grand Business Inn,"2/4, 2/11 Sukhumvit 11, Sukhumvit Road, Wattan...",None,8.0,BTS-Nana 0.1 km; Embassy of Sweden 0.2 km; Suk...,Non-smoking rooms; Free WiFi; Restaurant; 24-h...,"Wednesday, February 20, 2019","Thursday, February 21, 2019"
0,3,Anantara Baan Rajprasong Bangkok Serviced Suites,"3 Soi Mahardlekluang 3, Rajdamri Road, Lumpini...",5 stars,8.9,BTS-Rachadamri 0.1 km; CIMB Thai Bank Head Off...,Swimming pool; Outdoor pool; Airport shuttle; ...,"\nWednesday, February 20, 2019\n","\nThursday, February 21, 2019\n"
0,4,De Lavender Bangkok Hotel,"122/73, Ratchaprarop Rd, Phaya Thai Rd., Pratu...",3-star hotel,6.8,Baiyoke Sky Tower 0.1 km; Shibuya 19 0.1 km; P...,Airport shuttle; Non-smoking rooms; Spa; Famil...,"Wednesday, February 20, 2019","Thursday, February 21, 2019"
0,5,Le Meridien Bangkok,"40/5 Surawong Road, Silom, Bang Rak, 10500 Ban...",5-star hotel,9.0,Patpong 0.2 km; Bangkok Christian Hospital 0.2...,Swimming pool; Outdoor pool; Airport shuttle; ...,"\nWednesday, February 20, 2019\n","\nThursday, February 21, 2019\n"
0,6,The Emerald Hotel,"99/1 Rachadapisek Road, Huai Khwang, Din Daeng...",4-star hotel,7.8,Bangkok International Dental Center 0.3 km; MR...,Swimming pool; Outdoor pool; Non-smoking rooms...,"\nWednesday, February 20, 2019\n","\nThursday, February 21, 2019\n"
0,7,Best Western Premier Sukhumvit,"78 Sukhumvit Soi 1, Klongtoey Nua, Bangkok, Wa...",4-star hotel,8.2,Bumrungrad International Hospital 0.1 km; Krun...,Swimming pool; Outdoor pool; Non-smoking rooms...,"Wednesday, February 20, 2019","Thursday, February 21, 2019"
0,8,Holiday Inn Express Bangkok Sukhumvit 11,"30 Sukhumvit Soi 11, Klongtoey-Nua, Wattana, W...",3-star hotel,7.9,Sukhumvit Soi 11 0.3 km; Arab Street 0.5 km; B...,Non-smoking rooms; Fitness center; Family room...,"Wednesday, February 20, 2019","Thursday, February 21, 2019"
0,9,Kc Place Hotel Pratunam,"178/35 Rachaparop Soi 5, Rachaparop Road, Path...",3-star hotel,7.9,Ratchaprarop Station 0.1 km; Baiyoke Sky Tower...,Non-smoking rooms; Family rooms; Free WiFi; 24...,"Wednesday, February 20, 2019","Thursday, February 21, 2019"


In [311]:
df_all_room.to_csv('all_room_booking.csv')
df_all_room

,hid,roomName,bedType,capacity,rackPrice,price,breakfast,features
0,0,2 Beds in 8-Bed Dormitory Room,2 bunk beds,2,THB 902,THB 902,Wonderful breakfast included; Non-refundable,Air conditioning; Free WiFi; Linens
1,0,2 Beds in 8-Bed Dormitory Room,2 bunk beds,2,"THB 1,012","THB 1,012",Wonderful breakfast included\n\n\n\nNon-refund...,Air conditioning; Free WiFi; Linens
2,0,2 Beds in 4-Bed Dormitory Room,2 bunk beds,2,"THB 1,012","THB 1,012",Wonderful breakfast included; Non-refundable,Air conditioning; Free WiFi; Shower; Hairdryer...
3,0,2 Beds in 4-Bed Dormitory Room,2 bunk beds,2,"THB 1,122","THB 1,122",Wonderful breakfast included\n\n\n\nNon-refund...,Air conditioning; Free WiFi; Shower; Hairdryer...
4,0,Double Room with Shared Bathroom,1 full bed,2,"THB 1,122","THB 1,122",Wonderful breakfast included; Non-refundable,Air conditioning; Free WiFi; Linens; More
5,0,Double Room with Shared Bathroom,1 full bed,2,"THB 1,885.95","THB 1,885.95",Wonderful breakfast included\n\n\n\nNon-refund...,Air conditioning; Free WiFi; Linens; More
6,0,Quadruple Room with Shared Bathroom,4 bunk beds,4,"THB 1,885.95","THB 1,885.95",Wonderful breakfast included; Non-refundable,Air conditioning; Free WiFi; Linens; More
7,0,Quadruple Room with Shared Bathroom,4 bunk beds,4,"THB 2,592.86","THB 2,592.86",Wonderful breakfast included\n\n\n\nNon-refund...,Air conditioning; Free WiFi; Linens; More
8,0,6-Bed Mixed Dormitory Room,6 bunk beds,6,"THB 2,592.86","THB 2,592.86",Wonderful breakfast included; Non-refundable,Air conditioning; Free WiFi; Linens; More
9,0,6-Bed Mixed Dormitory Room,6 bunk beds,6,"THB 3,339.13","THB 3,339.13",Wonderful breakfast included\n\n\n\nNon-refund...,Air conditioning; Free WiFi; Linens; More


In [315]:
import sys

h = hash('Grand Business Inn') + sys.maxsize + 1
print(h)

8383728632389484888


In [316]:
h2 = hash('Grand Business Inn') + sys.maxsize + 1
print(h2)

8383728632389484888
